In [1]:
import os
from glob import glob
import shutil
import json
from tqdm import tqdm
import pandas as pd
from pathlib import Path
import jsonlines
import hashlib
import re
from PIL import Image
src = "/home/kai/workspace/DeepDocs_Project/datalake/source/dataset=post_handwritten_plain_text"

In [2]:
images = glob(os.path.join(src, "images", "*.*g"))
jsonl_path = glob(os.path.join(src, "*.jsonl"))[0]

In [3]:
jsonl_path

'/home/kai/workspace/DeepDocs_Project/datalake/source/dataset=post_handwritten_plain_text/metadata.jsonl'

In [4]:
with jsonlines.open(jsonl_path, mode='r') as reader:
    data = [obj for obj in reader]

In [6]:
import numpy as np
from PIL import Image
def get_safe_image_hash(path):
    img = Image.open(path).convert("RGB")
    arr = np.array(img)
    meta = f"{arr.shape}{arr.dtype}".encode()
    return hashlib.sha256(arr.tobytes() + meta).hexdigest()

def get_safe_image_hash_from_pil(img: Image.Image) -> str:
    arr = np.array(img.convert("RGB"))
    meta = f"{arr.shape}{arr.dtype}".encode()
    return hashlib.sha256(arr.tobytes() + meta).hexdigest()

In [7]:
records = []
MAX_SIZE = 2048
os.makedirs("images", exist_ok=True)
for d_ in tqdm(data):
    file_name = d_['file_name']
    gts = json.loads(d_['ground_truth'])['gt_parse']['주소 정보']
    label = {"address": []}
    for gt_ in gts:
        gt = gt_[list(gt_.keys())[0]]
        text = gt['도로명 주소'].strip()
        label['address'].append({
            "<|value|>": text,
        })
    if not os.path.exists(os.path.join(src, "images", file_name)):
        print(f"Image {file_name} not found in source directory.")
        continue
    image = Image.open(os.path.join(src, "images", file_name))
    
    width, height = image.size
    scale = min(MAX_SIZE / width, MAX_SIZE / height)
    if scale < 1:
        new_size = (int(width * scale), int(height * scale))
        image = image.resize(new_size, Image.LANCZOS)
    width, height = image.size
    hash = get_safe_image_hash_from_pil(image)
    image_basename = hash + ".jpg"
    image.save(os.path.join("images", image_basename), quality=95, subsampling=0)
    records.append({
        "image_path": image_basename,
        "width": width,
        "height": height,
        "label": json.dumps(label, ensure_ascii=False),
    })

  0%|          | 0/507 [00:00<?, ?it/s]

 11%|█         | 57/507 [00:10<01:00,  7.42it/s]

Image 0000.jpg not found in source directory.


100%|██████████| 507/507 [01:51<00:00,  4.55it/s]


In [8]:
len(os.listdir("images"))

506

In [9]:
df = pd.DataFrame(records)

In [10]:
df.to_parquet("data.parquet", index=False)